In [25]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

## Load Data Files

In [26]:
adEvents  = pd.read_csv("../Data/Raw Data/adevents.csv", sep=",", index_col='DEIDNUM')
quality = pd.read_csv("../Data/Raw Data/quality.csv", sep=",", index_col='DEIDNUM')
patient  = pd.read_csv("../Data/Raw Data/patient.csv", sep=",", index_col='DEIDNUM')
mhist  = pd.read_csv("../Data/Raw Data/mhist.csv", sep=",", index_col='DEIDNUM')
physical  = pd.read_csv("../Data/Raw Data/physical.csv", sep=",", index_col='DEIDNUM')

hemoOrig = pd.read_csv("../Data/Raw Data/hemo.csv", sep=",", index_col='DEIDNUM')

ex = pd.read_csv("../Data/Raw Data/exercise.csv", sep=",", index_col='DEIDNUM')
labs = pd.read_csv("../Data/Raw Data/labs.csv", sep=",", index_col='DEIDNUM')
meds = pd.read_csv("../Data/Raw Data/meds.csv", sep=",", index_col='DEIDNUM').fillna(0)

## Make Outcome Labels

In [27]:
dthList =  []
rhspList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    #Death outcome
    dth = t["DEATH"] + t['LVADANY'] + t['TRANSP']
    if dth >= 1:
        dthList.append(1)
    else:
        dthList.append(0)
        
    #Rehosp outcome
    rhsp = t['REHOSP']
    if rhsp >= 1:
        rhspList.append(1)
    else:
        rhspList.append(0)
        

#make readmit labels
reAdList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    adt = t['T2REHSP']  #days to first rehosp
    if adt < 30:
        reAdList.append(1)
    else:
        reAdList.append(0)

In [28]:
labels = pd.DataFrame(columns = ['Death', 'Rehosp', 'Readmission'], index=patient.index)
labels['Death'] = dthList
labels['Rehosp'] = rhspList
labels['Readmission'] = reAdList
labels.head(10)

,Death,Rehosp,Readmission
DEIDNUM,,,
72,1,1,0
81,0,1,0
86,0,0,0
267,1,0,0
814,0,0,0
1020,0,0,0
1059,1,1,0
1084,0,0,0
1262,0,1,0


In [29]:
print( "Death/LVAD/Trans Count", len(labels[labels['Death']==1]))
print("Rehosp Count", len(labels[labels['Rehosp']==1]))
print("Readmit Count", len(labels[labels['Readmission']==1]))

labels.to_csv('../Data/Original DataFrames/Labels.csv', index=True)

Death/LVAD/Trans Count 117
Rehosp Count 247
Readmit Count 80


## Make Meds

In [30]:
grandList = []
idx = sorted(set(patient.index))
for i in idx:
    lst = []
    t = meds.loc[i]
    #Ace
    aceB = t["ACEB"] + t['BENB'] + t['LISB'] + t['MONB'] + t['VALSAB'] + t['ANGIOTB'] + t['LOSAB'] + t['CANDB'] + t['CAPB'] + t['ENAB'] + t['FOSB'] + t['QUIB'] + t['RAMB'] + t['TRAB']  
    aceD = t["ACED"] + t['BEND'] + t['LISD'] + t['MOND'] + t['VALSAD'] + t['ANGIOTD'] + t['LOSAD'] + t['CANDD'] + t['CAPD'] + t['ENAD'] + t['FOSD'] + t['QUID'] + t['RAMD'] + t['TRAD']
    lst.append(1 if aceB >= 1 else 0)
    lst.append(1 if aceD >= 1 else 0)
    
    #BET
    betB = t['BETB'] + t['BISB'] + t['CARB'] + t['ATEB'] + t['PROB']
    betD = t['BETD'] + t['BISD'] + t['CARD'] + t['ATED'] + t['PROD']
    lst.append(1 if betB >= 1 else 0)
    lst.append(1 if betD >= 1 else 0)
    
    #NIT
    nitB = t['NITB'] + t['DINB'] + t['NIPB'] + t['BUMB'] + t['ETHB'] + t['NIGB'] + t['TOPB']
    nitD = t['NITD'] + t['DIND'] + t['NIPD'] + t['BUMD'] + t['ETHD'] + t['NIGD'] + t['TOPD']
    lst.append(1 if nitB >= 1 else 0)
    lst.append(1 if nitD >= 1 else 0)
    
    #Diuretics
    diurB = t['FURB'] + t['TORB'] + t['METB'] + t['DIURB'] 
    diurD = t['FURD'] + t['TORD'] + t['METD'] + t['DIURD'] 
    lst.append(1 if diurB >= 1 else 0)
    lst.append(1 if diurD >= 1 else 0)
    
    grandList.append(lst)


cols = ['ACE_B', 'ACE_D', 'BET_B', 'BET_D', 'NIT_B', 'NIT_D', 'DIUR_B', 'DIUR_D']
medData = pd.DataFrame(grandList, columns =cols, index=patient.index)

medData.head(10)

,ACE_B,ACE_D,BET_B,BET_D,NIT_B,NIT_D,DIUR_B,DIUR_D
DEIDNUM,,,,,,,,
72,1,1,1,0,1,1,1,1
81,1,1,1,1,0,1,1,1
86,1,1,0,0,1,1,1,1
267,1,1,1,0,0,1,1,1
814,0,1,1,1,0,0,1,1
1020,1,1,0,1,1,1,1,1
1059,1,1,0,0,1,0,1,1
1084,1,1,0,1,0,1,1,1
1262,1,1,0,1,1,1,1,1


## Make Lab Data

In [31]:
labUnits = pd.read_csv("../Data/Raw Data/labUnits.csv", index_col='DEIDNUM')

labUnits

,FORM,VISIT,PAGEID,PAGEREP,CRFPAGE,PERIOD,LAB,LABVAL,LABUNIT,LABDAY,LABDT
DEIDNUM,,,,,,,,,,,
75134,6-Month Follow-Up,NaN,68,NaN,67,8.0,1,NaN,NaN,NaN,176.0
75134,6-Month Follow-Up,NaN,68,NaN,67,8.0,2,NaN,NaN,NaN,176.0
75134,6-Month Follow-Up,NaN,68,NaN,67,8.0,3,NaN,NaN,NaN,176.0
75134,6-Month Follow-Up,NaN,68,NaN,67,8.0,4,NaN,NaN,NaN,176.0
75134,6-Month Follow-Up,NaN,68,NaN,67,8.0,5,141.0,8.0,NaN,176.0
...,...,...,...,...,...,...,...,...,...,...,...
49241,6-Month Follow-Up,NaN,68,NaN,67,8.0,10,NaN,NaN,NaN,163.0
49241,6-Month Follow-Up,NaN,68,NaN,67,8.0,11,NaN,NaN,NaN,163.0
49241,6-Month Follow-Up,NaN,68,NaN,67,8.0,12,NaN,NaN,NaN,163.0


In [ ]:
#go thru lab vals (alb - wbc) using code nums
#then get units and convert to be standardized (ref crf annotations)

base = []
final = []

idx = sorted(set(labs.index))
for i in idx:
    pt = labUnits.loc[i]
    
    #get baseline
    pt = pt[pt['FORM'] == 'Baseline']
    pt = pt[pt['LAB'] == 12] #get alb lab val
    
    print(pt)
    unit = pt['LABUNIT'].item()
    
    print("unit is", unit)
    

In [ ]:
labData = pd.DataFrame(columns =[], index=patient.index)

var = ['ALB', 'ALT', 'AST', 'BUN', 'CRT', 'DIAL', 'HEC', 'HEM', 'PLA', 'POT', 'SOD', 'TALB', 'TOTP', 'WBC']

for v in var:
    idx = sorted(set(labs.index))
    base = []
    final = []
    for i in idx:
        t = labs.loc[i]
       
        base.append(t[v+"B"])
        final.append(t[v+"D"])
        
    labData[v+"_B"] = base
    labData[v+"_D"] = final
    
labData.head(5)

## Make Conditions

In [ ]:
grandList = []
idx = sorted(set(patient.index))
for i in idx:
    lst = []
    t = mhist.loc[i]

    lst.append(t['AFIB'])
    lst.append(t['ALCHOE'])
    lst.append(t['ANGP'])
    lst.append(t['ARRH'])
    lst.append(t['CARREST'])
    lst.append(t['CVD'])
    lst.append(t['COPD'])
    lst.append(t['DEPR'])
    lst.append(t['DIAB'])
    lst.append(t['GOUT'])
    lst.append(t['HEPT'])
    lst.append(t['HTN'])
    lst.append(t['MALIG'])
    lst.append(t['RENALI'])
    lst.append(t['SMOKING'])
    lst.append(t['STERD'])
    
    #Stroke / TIA
    if t['STROKE'] == 1 or t['TIA'] == 1:
        lst.append(1)
    else:
        lst.append(0)
    
    #VAHD
    if t['VAHD'] == 1 or t['PVD'] == 1:
        lst.append(1)
    else:
        lst.append(0)

    lst.append(t['VF'])
    
    #VHD
    vhd = t['AOREG'] + t['AOST'] + t['MTST'] + t['PMRG'] + t['PTREG'] + t['VALVUE']
    lst.append(1 if vhd >= 1 else 0)
    
    #VT
    vt = t['SVT'] + t['TDP']
    lst.append(1 if vt >= 1 else 0)
    
    #ischemic
    isch = t['ISCHD'] + t['ISCHEME'] + t['MI'] + t['CYTOE']
    lst.append(1 if isch >= 1 else 0)
    
    #nonischemic
    nisch = t['FAMILE'] + t['HYPERE'] + t['IDIOPE'] + t['PERIPAE']
    lst.append(1 if nisch >= 1 else 0)
    
    #Devices / procedures
    lst.append(t['CABG'])
    lst.append(1 if t['HTRANS'] >= 1 else 0)
    lst.append(t['ICD'])
    lst.append(t['PACE'])
    lst.append(t['PTCI'])
    
    grandList.append(lst)
    


cols = ['AF', 'AlchE','ANGP','ARRH','CARREST','CVD','COPD','DEPR','DIAB','GOUT','HEPT','HTN','MALIG','RENAL','SMOKING',
        'STERD','StrokeTIA','VAHD','VF','VHD','VT', 'ISCH', 'NonISCH', 'CABG', 'HTRANS', 'ICD', 'PACE', 'PTCI']
conditions = pd.DataFrame(grandList, columns =cols, index=patient.index)

conditions.head(10)

## All Data

In [ ]:
#  no hemo, just bps
allData = pd.DataFrame(columns =[], index=patient.index)

#demog
allData['Age'] = patient['AGE']
allData['Gender'] = patient['GENDER']
allData['Race'] = patient['RACE']
allData['Wt_B'] = patient['WTADM']
allData['Wt_D'] = patient['WTDIS']
allData['BMI_B'] = patient['bmiadm']
allData['BMI_D'] = patient['bmidis']

allData['InitialHospDays'] = patient['HOSPDAY']
allData['TotalHospDays'] = patient['HSPDAY']

allData['NYHA_B'] = quality['NYHAB']
allData['NYHA_D'] = quality['NYHAD']
allData['MLHFS'] = quality['MLHFSB']


#conditions
allData = pd.concat([allData, conditions], axis=1)

#ex
allData['SixFtWlk_B'] = ex['FTWLKB']
allData['SixFtWlk_D'] = ex['FTWLKD']
allData['VO2_B'] = ex['VO2B']
allData['VO2_D'] = ex['VO2D']

#labs
allData = pd.concat([allData, labData], axis=1)

# meds
allData = pd.concat([allData, medData], axis=1)


# #Hemo ish
allData['EjF_B'] = patient['EF1']
allData['EjF_D'] = patient['EF1']
allData['BPDIAS_B'] = physical['DIASBPB']
allData['BPDIAS_D'] = physical['DIASBPD']
allData['BPSYS_B'] = physical['SYSBPB']
allData['BPSYS_D'] = physical['SYSBPD']
allData['HR_B'] = physical['HRSUPB']
allData['HR_D'] = physical['HRSUPD']


# #Composite
idx = sorted(set(allData.index))
#PV
pv = []
for i in idx:
    t = allData.loc[i]
    
    val = ( ((t['HEM_B'] / t['HEM_D']) * ((100 - t['HEC_D']) / (100  - t['HEC_B']))) - 1 ) * 100
    pv.append(val)
    
allData['PV'] = pv

#MAP
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['MAP_B'] = base
allData['MAP_D'] = final
allData['MAP_Chg'] = chg

#PP
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['PP_B'] = base
allData['PP_D'] = final
allData['PP_Chg'] = chg

#PPP
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['PPP_B'] = base
allData['PPP_D'] = final
allData['PPP_Chg'] = chg

#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['PP_B'] / t['HR_B']
    f = t['PP_D'] / t['HR_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['PPRatio_B'] = base
allData['PPRatio_D'] = final
allData['PPRatio_Chg'] = chg


allData.head(5)

In [ ]:
allData.to_csv('../Data/Original DataFrames/AllData.csv', index=True)

## Hemo

In [ ]:
#Calculate hemo changes
hemo = pd.DataFrame(columns = ['RAP_B', 'PAS_B', 'PAD_B', 'PAMN_B', 'PCWP_B',
                               'CO_B', 'CI_B', 'SVR_B', 'MIXED_B', 'BPSYS_B', 'BPDIAS_B', 'HRTRT_B', 'RAT_B',
                               'RAP_D', 'PAS_D', 'PAD_D', 'PAMN_D', 'PCWP_D',  
                               'CO_D', 'CI_D', 'SVR_D', 'MIXED_D', 'BPSYS_D', 'BPDIAS_D', 'HRTRT_D', 'RAT_D',
                               'RAP_Chg', 'PAS_Chg', 'PAD_Chg', 'PAMN_Chg', 'PCWP_Chg', 
                               'CO_Chg', 'CI_Chg', 'SVR_Chg', 'MIXED_Chg', 'BPSYS_Chg', 'BPDIAS_Chg', 
                               'HRTRT_Chg', 'RAT_Chg'], index=patient.index)


var = ['RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 'CI', 'SVR', 'MIXED', 'BPSYS', 
       'BPDIAS', 'HRTRT', 'RAT']

for v in var:
    idx = sorted(set(hemoOrig.index))
    chg = []
    base = []
    final = []
    for i in idx:
        t = hemoOrig.loc[i]
        
        base.append(t[v+"B"])
        final.append(t[v+"L"])
        chg.append(t[v+"L"] - t[v+"B"])
        
    hemo[v+"_B"] = base
    hemo[v+"_D"] = final
    hemo[v+"_Chg"] = chg
    
hemo.head(5)

In [ ]:
#Get only patients with hemo
hemoComposite = hemo.loc[~(np.isnan(hemo)).all(axis=1)]
print(hemoComposite.shape)
hemoComposite.head(5)

In [ ]:
#Add demog - age, gender, race, ejf
idx = sorted(set(hemoComposite.index))

age = []
gender = []
race = []
ejf = []
for i in idx:
    t = hemoComposite.loc[i]
    age.append(patient.loc[i]['AGE'])
    gender.append(patient.loc[i]['GENDER'])
    race.append(patient.loc[i]['RACE'])
    ejf.append(patient.loc[i]['EF1'])

hemoComposite['Age_B'] = age
hemoComposite['Age_D'] = age
hemoComposite['Gender_B'] = gender
hemoComposite['Gender_D'] = gender
hemoComposite['Race_B'] = race
hemoComposite['Race_D'] = race
hemoComposite['EjF_B'] = ejf
hemoComposite['EjF_D'] = ejf

hemoComposite.head(5)


In [ ]:
var = ['MAP', 'MPAP', 'CPI', 'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio']

idx = sorted(set(hemoComposite.index))

#MAP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['MAP_B'] = base
hemoComposite['MAP_D'] = final
hemoComposite['MAP_Chg'] = chg

#MPAP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PAS_B'] + ((2 * t['PAD_B']) / 3)
    f = t['PAS_D'] + ((2 * t['PAD_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['MPAP_B'] = base
hemoComposite['MPAP_D'] = final
hemoComposite['MPAP_Chg'] = chg

#CPI
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['CI_B'] * t['MAP_B']) / 451
    f = (t['CI_D'] * t['MAP_D']) / 451
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['CPI_B'] = base
hemoComposite['CPI_D'] = final
hemoComposite['CPI_Chg'] = chg
    
#PP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PP_B'] = base
hemoComposite['PP_D'] = final
hemoComposite['PP_Chg'] = chg

#PPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PPP_B'] = base
hemoComposite['PPP_D'] = final
hemoComposite['PPP_Chg'] = chg
    
#PAPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['PAS_B']  - t['PAD_B']) / t['PAS_B']
    f = (t['PAS_D']  - t['PAD_D']) / t['PAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PAPP_B'] = base
hemoComposite['PAPP_D'] = final
hemoComposite['PAPP_Chg'] = chg

#SVR
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = 80 * (t['MAP_B']  - t['RAP_B']) / t['CO_B']
    f = 80 * (t['MAP_D']  - t['RAP_D']) / t['CO_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['SVR_B'] = base
hemoComposite['SVR_D'] = final
hemoComposite['SVR_Chg'] = chg

#RAT
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['RAP_B'] / t['PCWP_B']
    f = t['RAP_D'] / t['PCWP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['RAT_B'] = base
hemoComposite['RAT_D'] = final
hemoComposite['RAT_Chg'] = chg

#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['HRTRT_B']
    f = t['PP_D'] / t['HRTRT_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PPRatio_B'] = base
hemoComposite['PPRatio_D'] = final
hemoComposite['PPRatio_Chg'] = chg

#PAPi
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['PAS_B'] - t['PAD_B']) / t['RAP_B']
    f = (t['PAS_D'] - t['PAD_D']) / t['RAP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PAPi_B'] = base
hemoComposite['PAPi_D'] = final
hemoComposite['PAPi_Chg'] = chg

#SAPi
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['BPSYS_B'] - t['BPDIAS_B']) / t['PCWP_B']
    f = (t['BPSYS_D'] - t['BPDIAS_D']) / t['PCWP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['SAPi_B'] = base
hemoComposite['SAPi_D'] = final
hemoComposite['SAPi_Chg'] = chg

#CPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPDIAS_B'] - t['PCWP_B']
    f = t['BPDIAS_B'] - t['PCWP_B']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['CPP_B'] = base
hemoComposite['CPP_D'] = final
hemoComposite['CPP_Chg'] = chg

#PRAPRat
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['RAP_B']
    f = t['PP_D'] / t['RAP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PRAPRat_B'] = base
hemoComposite['PRAPRat_D'] = final
hemoComposite['PRAPRat_Chg'] = chg

hemoComposite.head(5)

In [ ]:
hemoComposite.to_csv('../Data/Original DataFrames/HemoComposite.csv', index=True)

## Make single point of care values
--> take base and discharge measurements and make them each one row

In [ ]:
#Load original dataframes
allDataOrig = pd.read_csv("../Data/Original DataFrames/AllData.csv", sep=",", index_col='DEIDNUM') #all feature dataset
hemoOrig = pd.read_csv("../Data/Original DataFrames/HemoComposite.csv", sep=",", index_col='DEIDNUM') #dataset with only hemodynamics

In [ ]:
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(hemoOrig)):
    df = hemoOrig.iloc[i]
    row0 = []
    row1 = []
    
    row0.append(df.name)
    row1.append(df.name)
    for v in varz:
    

        row0.append(df[v+"_B"])
        row1.append(df[v+"_D"])

        
    giantList.append(row0)
    giantList.append(row1)


varz.insert(0, 'DEIDNUM')
hemoSingleDF = pd.DataFrame(giantList, columns=varz).set_index('DEIDNUM', drop=True)

In [ ]:
hemoSingleDF.head(8)

In [ ]:
#save to file
hemoSingleDF.to_csv("../Data/Original DataFrames/HemoSingleValue.csv")

## All Data

In [ ]:
giantList = []
for i in range(len(allDataOrig)):
    df = allDataOrig.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in df.index:
        if "_B" in col:
            row0.append(df[col])
        elif "_D" in col:
            row1.append(df[col])
        elif "_Chg" in col or "_M3" in col or "_M6" in col:
            pass
        else: #value for both base and discharge
            row0.append(df[col])
            row1.append(df[col])
    
    giantList.append(row0)
    giantList.append(row1)

    

allDataSingleDF = pd.DataFrame(giantList, columns=['DEIDNUM','Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']
).set_index('DEIDNUM', drop=True)

In [ ]:
allDataSingleDF.head(8)

In [ ]:
#save to file
allDataSingleDF.to_csv("../Data/Original DataFrames/AllDataSingleValue.csv")